In [13]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
from PyPDF2 import PdfReader

In [14]:
# prompt: get gemini api key and create a client connection to gemini

# imports

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

google = genai.Client(api_key=google_api_key)


In [15]:
def modifyImage(image, prompt):
    response = google.models.generate_content(
        model="gemini-2.0-flash-preview-image-generation",
        contents=[prompt, image],
        config=types.GenerateContentConfig(
            response_modalities=['TEXT', 'IMAGE'],
            temperature=0.1
        )
    )

    for part in response.candidates[0].content.parts:
      if part.text is not None:
        print(part.text)
      elif part.inline_data is not None:
        image = Image.open(BytesIO(part.inline_data.data))
        #image.show()
    return image

In [16]:
styled_image = modifyImage(Image.open('my_picture.jpg'), 'convert the attached images to a Ghibli art work.')

I will transform the photograph into a vibrant Ghibli-style illustration. The scene will depict a man in a blue striped shirt and light blue shorts standing on a balcony with a glass railing. Behind him, the iconic Hollywood sign will be visible on a sun-drenched, grassy hillside under a clear blue sky, rendered with the characteristic soft lines and lush colors of Studio Ghibli animation.




In [17]:
def read_pdf(file_name):
    file = open(file_name, "rb")
    bin = file.read()
    reader = PdfReader(BytesIO(bin))

    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text
    
def find_companies(text):
  response = google.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction="You are a career assistant."),
    contents=('From the following text, please identity the company names I worked at or served as client. Please only return the company names, with each company name on a new line, no other text: \n' + text)
  )
  return response.candidates[0].content.parts[0].text

In [18]:
pdf_text = read_pdf('my_profile.pdf')
company_response = find_companies(pdf_text)

In [19]:
final_image = modifyImage(styled_image, 'Please add the following company logos to the background, make sure only add each logo once:\n' + company_response)

I will add the specified company logos as subtle elements in the background of the image, ensuring each logo appears only once and does not distract from the main subject.




In [20]:
final_image.save('ai_profile_picture.png')